In [1]:
import pandas as pd
import numpy as np
import random
from quantnote import BacktestEngine, Strategy

In [2]:
db_path = '/Users/j.nahmgoong/Documents/git/DB/MainDB.db'

engine = BacktestEngine(db_path)

Loading DB...
DB loaded in 48.10 seconds
[initialize] is executed in 57.07 seconds


In [3]:
non_cap_norm = ['tbvps',
'sps',
'ros',
'roe',
'roa',
'ps1',
'ps',
'price',
'prefdivis',
'pe1',
'pe',
'pb',
'payoutratio',
'fcfps',
'epsdil',
'eps',
'dps',
'divyield',
'de',
'currentratio',
'bvps',
'marketcap']

In [4]:
class myStrategy(Strategy):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        print(self.config)

    def compute_target(self, universe_list):
        target_weight = { }
        
        input1 = self.config['input1']
        input2 = self.config['input2']
        unary = self.config['unary']
        binary = self.config['binary']
        unary_out = self.config['unary_out']
        
        factor_series_1 = self.compute_factor_series(universe_list, input1, type=unary)
        factor_series_2 = self.compute_factor_series(universe_list, input2, type=unary)
        
        if binary == 'add':
            factor_series = factor_series_1 + factor_series_2
        elif binary == 'sub':
            factor_series = factor_series_1 - factor_series_2
        elif binary == 'mul':
            factor_series = factor_series_1 * factor_series_2
        elif binary == 'div':
            factor_series = factor_series_1 / factor_series_2
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        assert len(factor_series) > 0.5*len(universe_list)
        
        if unary_out == 'raw':
            pass
        elif unary_out == 'rank':
            factor_series = factor_series.rank(ascending=False)
        elif unary_out == 'zscore':
            factor_series = (factor_series-factor_series.mean())/factor_series.std()
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        top = int(len(factor_series.index)/5)
        
        for ticker in factor_series.index:
            weight = factor_series[ticker]
            if weight > 0:
                target_weight[ticker] = weight
                
        assert len(target_weight) >= 25
        
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            if ftype in non_cap_norm:
                x = self.get_value('fundamentals',ticker,ftype)
            else:
                x = self.get_value('fundamentals',ticker,ftype)/self.get_value('fundamentals',ticker,'marketcap')
            return x



In [8]:
input_list = list(engine.cache['fundamentals']['AAPL'].columns[5:])
unary_list = ['raw', 'rank', 'zscore']
binary_list = ['add', 'sub', 'mul', 'div']

# result_df = pd.DataFrame({
#     'sharpe':[],
#     'cagr':[],
#     'vol':[],
#     'input1':[],
#     'input2':[],
#     'unary':[],
#     'binary':[],
#     'unary_out':[],
# })
result_df = pd.read_parquet('result.parquet')
len(input_list)

105

In [ ]:
idx = 1
while True:
    x1, x2 = random.sample(input_list,2)
    unary = random.sample(unary_list,1)[0]
    binary = random.sample(binary_list,1)[0]
    unary_out = random.sample(unary_list,1)[0]
    
    config = {
    'input1':x1,
    'input2':x2,
    'unary':unary,
    'binary':binary,
    'unary_out':unary_out}
    
    print('\n')
    print('Computing {}th factor'.format(idx))
    mys = myStrategy(config)
    try:
        engine.run_backtest(mys, '2016-01-01','2021-08-31',period='Q')
        engine.stat(bench=None)
        print(engine.stat_dic)
        
        sharpe = engine.stat_dic['sharpe']
        cagr = engine.stat_dic['cagr']
        vol = engine.stat_dic['vol']
        
        result_df = result_df.append({
            'sharpe':sharpe,
            'cagr':cagr,
            'vol':vol,
            'input1':x1,
            'input2':x2,
            'unary':unary,
            'binary':binary,
            'unary_out':unary_out,
        },ignore_index=True)
        
        result_df.to_parquet('result.parquet')

        idx += 1
    except:
        print('Passed!')
        
    result_df = result_df.sort_values('sharpe', ascending=False)
    print('Current best sharpe')
    try:
        print(result_df.iloc[:3])
    except:
        pass




Computing 1th factor
{'input1': 'shareswa', 'input2': 'roa', 'unary': 'raw', 'binary': 'div', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00


C:\Users\j.nahmgoong\Anaconda3\lib\site-packages\quantnote\strategy.py:73: RuntimeWarning: invalid value encountered in double_scalars
  target_weight = {ticker:target_weight[ticker]/target_sum for ticker in target_weight}


Passed!
Current best sharpe
     sharpe      cagr       vol      input1       input2 unary binary  \
0  0.981191  0.188063  0.191668   invcapavg        fxusd  rank    sub   
1  0.845892  0.156401  0.184895        ncfi       ebitda  rank    sub   
2  0.811781  0.150171  0.184989  revenueusd  grossmargin   raw    div   

  unary_out  
0       raw  
1      rank  
2      rank  


Computing 1th factor
{'input1': 'ncf', 'input2': 'sgna', 'unary': 'rank', 'binary': 'div', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.558 / time elapsed:66.1 ===
[run_backtest] is executed in 66.48 seconds
{'cagr': 0.07727444900729823, 'vol': 0.19881294011979958, 'sharpe': 0.38867917229499566}
Current best sharpe
     sharpe      cagr       vol      input1       input2 unary binary  \
0  0.981191  0.188063  0.191668   invcapavg        fxusd  rank    sub   
1  0.845892  0.156401  0.184895        ncfi       ebitda  rank    sub   
2 

=== date:2021-08-31 00:00:00 / total_asset:1.398 / time elapsed:69.1 ===
[run_backtest] is executed in 69.43 seconds
{'cagr': 0.05812275405455603, 'vol': 0.207778175762764, 'sharpe': 0.2797346441279721}
Current best sharpe
     sharpe      cagr       vol     input1       input2 unary binary unary_out
0  0.981191  0.188063  0.191668  invcapavg        fxusd  rank    sub       raw
1  0.845892  0.156401  0.184895       ncfi       ebitda  rank    sub      rank
2  0.825178  0.178132  0.215871        eps  payoutratio  rank    mul    zscore


Computing 9th factor
{'input1': 'sbcomp', 'input2': 'depamor', 'unary': 'rank', 'binary': 'mul', 'unary_out': 'rank'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.282 / time elapsed:137.3 ===
[run_backtest] is executed in 138.03 seconds
{'cagr': 0.14567095116014459, 'vol': 0.18844166869570095, 'sharpe': 0.7730294056957048}
Current best sharpe
     sharpe      cagr       vol     input1       input

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.218 / time elapsed:96.8 ===
[run_backtest] is executed in 97.52 seconds
{'cagr': 0.14075813424111358, 'vol': 0.19598441849042375, 'sharpe': 0.7182108420929972}
Current best sharpe
     sharpe      cagr       vol        input1         input2 unary binary  \
0  0.981191  0.188063  0.191668     invcapavg          fxusd  rank    sub   
1  0.878583  0.162747  0.185238  currentratio  investmentsnc   raw    sub   
2  0.845892  0.156401  0.184895          ncfi         ebitda  rank    sub   

  unary_out  
0       raw  
1    zscore  
2      rank  


Computing 17th factor
{'input1': 'shareswa', 'input2': 'liabilitiesc', 'unary': 'raw', 'binary': 'sub', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
Passed!
Current best sharpe
     sharpe      cagr       vol        input1         input2 unary binary  \
0  0.981191  0.188063  0.191668     invcapavg          fxus

...

=== date:2021-08-31 00:00:00 / total_asset:2.448 / time elapsed:106.2 ===
[run_backtest] is executed in 106.88 seconds
{'cagr': 0.15826327279153882, 'vol': 0.1847651911028727, 'sharpe': 0.8565643336109869}
Current best sharpe
     sharpe      cagr       vol     input1        input2   unary binary  \
0  1.052282  0.200718  0.190745       ebit        invcap     raw    div   
1  0.981191  0.188063  0.191668  invcapavg         fxusd    rank    sub   
2  0.911316  0.196747  0.215894     ncfdiv  currentratio  zscore    add   

  unary_out  
0       raw  
1       raw  
2    zscore  


Computing 31th factor
{'input1': 'sbcomp', 'input2': 'accoci', 'unary': 'zscore', 'binary': 'sub', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.865 / time elapsed:76.3 ===
[run_backtest] is executed in 76.74 seconds
{'cagr': 0.10841613359929721, 'vol': 0.21684676391830204, 'sharpe': 0.4999665738158927}
Current best sharpe
     sha

=== date:2021-08-31 00:00:00 / total_asset:2.047 / time elapsed:96.4 ===
[run_backtest] is executed in 96.99 seconds
{'cagr': 0.12609832502049337, 'vol': 0.17285909472424277, 'sharpe': 0.729486205059991}
Current best sharpe
     sharpe      cagr       vol     input1        input2   unary binary  \
0  1.052282  0.200718  0.190745       ebit        invcap     raw    div   
1  0.981191  0.188063  0.191668  invcapavg         fxusd    rank    sub   
2  0.911316  0.196747  0.215894     ncfdiv  currentratio  zscore    add   

  unary_out  
0       raw  
1       raw  
2    zscore  


Computing 40th factor
{'input1': 'ncfdebt', 'input2': 'accoci', 'unary': 'raw', 'binary': 'mul', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.632 / time elapsed:84.7 ===
[run_backtest] is executed in 85.26 seconds
{'cagr': 0.08525092089197892, 'vol': 0.23990460343582254, 'sharpe': 0.35535341827979805}
Current best sharpe
     sharpe   

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.066 / time elapsed:143.3 ===
[run_backtest] is executed in 144.05 seconds
{'cagr': 0.12821238959706055, 'vol': 0.1952060290495578, 'sharpe': 0.6568054799399189}
Current best sharpe
     sharpe      cagr       vol     input1        input2   unary binary  \
0  1.052282  0.200718  0.190745       ebit        invcap     raw    div   
1  0.981191  0.188063  0.191668  invcapavg         fxusd    rank    sub   
2  0.911316  0.196747  0.215894     ncfdiv  currentratio  zscore    add   

  unary_out  
0       raw  
1       raw  
2    zscore  


Computing 49th factor
{'input1': 'ebit', 'input2': 'equityavg', 'unary': 'zscore', 'binary': 'mul', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
Passed!
Current best sharpe
     sharpe      cagr       vol     input1        input2   unary binary  \
0  1.052282  0.200718  0.190745       ebit        invcap     raw    d

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.468 / time elapsed:138.3 ===
[run_backtest] is executed in 139.07 seconds
{'cagr': 0.0652646638318314, 'vol': 0.23015908590899614, 'sharpe': 0.2835632735248034}
Current best sharpe
     sharpe      cagr       vol     input1        input2   unary binary  \
0  1.052282  0.200718  0.190745       ebit        invcap     raw    div   
1  0.981191  0.188063  0.191668  invcapavg         fxusd    rank    sub   
2  0.911316  0.196747  0.215894     ncfdiv  currentratio  zscore    add   

  unary_out  
0       raw  
1       raw  
2    zscore  


Computing 58th factor
{'input1': 'ros', 'input2': 'pe1', 'unary': 'raw', 'binary': 'mul', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.931 / time elapsed:135.8 ===
[run_backtest] is executed in 136.65 seconds
{'cagr': 0.11610814321708131, 'vol': 0.20367938639393826, 'sharpe'

=== date:2021-08-31 00:00:00 / total_asset:1.826 / time elapsed:96.3 ===
[run_backtest] is executed in 96.84 seconds
{'cagr': 0.10481178751311533, 'vol': 0.2452192440002849, 'sharpe': 0.4274207268700069}
Current best sharpe
     sharpe      cagr       vol     input1        input2   unary binary  \
0  1.052282  0.200718  0.190745       ebit        invcap     raw    div   
1  0.981191  0.188063  0.191668  invcapavg         fxusd    rank    sub   
2  0.911316  0.196747  0.215894     ncfdiv  currentratio  zscore    add   

  unary_out  
0       raw  
1       raw  
2    zscore  


Computing 67th factor
{'input1': 'debtnc', 'input2': 'ncfbus', 'unary': 'raw', 'binary': 'add', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
Passed!
Current best sharpe
     sharpe      cagr       vol     input1        input2   unary binary  \
0  1.052282  0.200718  0.190745       ebit        invcap     raw    div   
1  0.981191  0.188063  0.191668  invcapavg         fxusd    

In [ ]:
result_df

In [ ]:
1+1